In [ ]:
from peft import LoraConfig, get_peft_model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from pathlib import Path
from datasets import load_dataset, Dataset
from trl import SFTTrainer
import pandas
from trainer.dataset_loader import create_random_message_stack_from_adjacent_records, create_message_stack, convert_message_stack_to_training_promots

### Check if CUDA is available

In [ ]:
torch.cuda.is_available()

## Load Model

In [ ]:
model_id = "C:/Users/h-alice/Desktop/llms/google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"":0})

outputs

In [ ]:
outputs

### Test Model

In [ ]:
text = """The reason that the sky has blue color:
"""

device = "cuda:0"
input_prompt = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**input_prompt, max_new_tokens=500, do_sample=True, top_k=20, repetition_penalty=1.5, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

## Prepare LoRA Config

In [ ]:
# For GEMMA model.
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# Smaller version.
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

## Load Dataset

In [ ]:
dataset_raw = load_dataset("h-alice/chat-cooking-master-boy-100k", split="train")
dataset_raw["message"][0:3]

In [ ]:
get_peft_model(model, lora_config).print_trainable_parameters()

In [ ]:
dataset_raw["message"][1:3]

In [ ]:
prompt_list = convert_message_stack_to_training_promots(
            create_message_stack(dataset_raw, batch_size=100, stack_size_min=2, stack_size_max=2, even_stack_size=True)) # type: ignore

In [ ]:
# Convert to pandas dataframe.
df = pandas.DataFrame(prompt_list, columns=["message"])

# Convert to huggingface dataset.
ds = Dataset.from_pandas(df)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        output_dir="outputs",
        optim="adamw"
    ),
    peft_config=lora_config,
    dataset_text_field="message",
)

In [ ]:
ds["message"]

In [ ]:
trainer.train()